# Baseline Model

For the baseline we have build a simple transformer that just has as an input context+question and tries to predict an answer

In [1]:
import pandas as pd
import numpy as np
import json

Helper class to covert json to dataframe for easier batch processing

In [2]:
class Squad:
    def __init__(self, input_location):
        self.location = input_location
        file = open(input_location)
        json_file = json.load(file)
        # Save version and data
        self.version = json_file['version']
        self.data = json_file['data']
        
        df_builder = [] # We will store every row of dataframe here
        for sample in self.data:
            title = sample['title'] # Get title
            paragraphs = sample['paragraphs']
            
            for paragraph in paragraphs:
                context = paragraph['context'] # Get context, e.g. a paragraph
                questions = paragraph['qas']
                
                for question in questions:
                    q_id = question['id'] # Question id
                    q_content = question['question'] # Question itself
                    answers = question['answers'] # Possible answers
                    is_impossible = question['is_impossible'] # If it is possible to answer
                    
                    # Build a row of dataframe
                    qas = {
                        'id':q_id,
                        'wiki_title':title,
                        'context':context,
                        'content':q_content,
                        'is_impossible':is_impossible
                    }
                    if is_impossible:
                        qas['answer'] = ""
                        qas['answer_start'] = -1
                    else:
                        answer = answers[0]
                        qas['answer'] = answer['text']
                        qas['answer_start'] = answer['answer_start']
                    df_builder.append(qas) 
        self.df = pd.DataFrame(df_builder)

In [3]:
train_sq = Squad('./data/train-v2.0.json')
test_sq = Squad('./data/dev-v2.0.json')
train_df  = train_sq.df
test_df  = test_sq.df

In [4]:
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [5]:
class BaselineModel(nn.Module):
    """A very simplistic model to predict answers"""
    def __init__(self, context_vocab, tgt_vocab, emb_size, num_head, num_encoder_layers=2,
                 num_decoder_layers=2, dim_feedforward=756, dropout=0.1, pad_token=1):
        super(BaselineModel, self).__init__()
        
        self.pad_token = pad_token
        self.context_emb = nn.Embedding(context_vocab, emb_size) # We embed context + question here
        
        self.tgt_emb = nn.Embedding(tgt_vocab, emb_size) # We embed target here
        
        encoder_layer = nn.TransformerEncoderLayer(emb_size, num_head, dim_feedforward, dropout, 'relu')
        encoder_norm = nn.LayerNorm(emb_size)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        decoder_layer = nn.TransformerDecoderLayer(emb_size, num_head, dim_feedforward, dropout, 'relu')
        decoder_norm = nn.LayerNorm(emb_size)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)
        
        # Linear layer to predict the output
        self.fc_out = nn.Linear(emb_size, tgt_vocab)
        self._init_params()
        self.tgt_vocab = tgt_vocab
    
    def forward(self, context, tgt=None, teacher_forcing=0.75):
        
        # Embed context and target
        context_embedded = self.context_emb(context)
        
        src_key_padding_mask = context == self.pad_token
        
        
        memory = self.encoder(context_embedded, mask=None, src_key_padding_mask=src_key_padding_mask.T)
        
        batch_size = context.shape[1]
        inp = torch.LongTensor([2]*batch_size).to(device).unsqueeze(0)
        
        outputs = None
        for i in range(1, TRG_LEN):
            inp_embedded = self.tgt_emb(inp)
            output = self.decoder(inp_embedded, memory)
            output_fc = model.fc_out(output)
            
            if outputs is None:
                outputs = output_fc
            else:
                outputs = torch.cat((outputs, output_fc), dim=0)
                
            inp = output.argmax(2)
            if np.random.random() < teacher_forcing and not len(tgt) <= i: # Try teaching forcing
                inp = tgt[i, :].unsqueeze(0)
        
        return outputs
    
    def _init_params(self):
        for p in self.parameters():
            if p.dim() > 1:
                torch.nn.init.xavier_uniform_(p)

In [6]:
from torchtext import *
from torchtext.data import *

# Taken from here for easier work with dataframe and torchtext
# https://gist.github.com/notnami/3c4d636f2b79e206b26acfe349f2657a
class DataFrameExampleSet:
    def __init__(self, df, fields):
        self._df = df
        self._fields = fields
        self._fields_dict = {field_name: (field_name, field)
                             for field_name, field in fields.items()
                             if field is not None}

    def __iter__(self):
        for item in tqdm(self._df.itertuples(), total=len(self)):
            example = Example.fromdict(item._asdict(), fields=self._fields_dict)
            yield example

    def __len__(self):
        return len(self._df)

    def shuffle(self, random_state=None):
        self._df = self._df.sample(frac=1.0, random_state=random_state)


class DataFrameDataset(Dataset):
    def __init__(self, df, fields, filter_pred=None):
        examples = DataFrameExampleSet(df, fields)
        super().__init__(examples, fields, filter_pred=filter_pred)


class DataFrameBucketIterator(BucketIterator):
    def data(self):
        if isinstance(self.dataset.examples, DataFrameExampleSet):
            if self.shuffle:
                self.dataset.examples.shuffle()
            dataset = self.dataset
        else:
            dataset = super().data()
        return dataset

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
# Concatenate content and context
train_df['content_q'] = train_df.context +' '+ train_df.content
test_df['content_q'] = test_df.context +' '+ test_df.content

In [9]:
import torchtext
from typing import *
from torchtext.data import *
from tqdm.notebook import tqdm
from torchtext.data.utils import get_tokenizer
import dill
load = False
TRG_LEN=15

if load:
    with open("model/CONTEXT_Q.Field","rb") as f:
        CONTEXT_Q=dill.load(f)
    with open("model/TRG.Field","rb") as f:
        TRG=dill.load(f)
else:
    # Init Fields 
    
    # Here will be context and question
    CONTEXT_Q = torchtext.data.Field(tokenize = get_tokenizer("basic_english"),
                          init_token = '<sos>',
                          eos_token = '<eos>',
                          lower = True,
                          batch_first = False)
    # here the target 
    TRG = torchtext.data.Field(tokenize = get_tokenizer("basic_english"), 
                         init_token = '<sos>',
                         eos_token = '<eos>',
                         lower = True,
                         batch_first = False,
                         fix_length=TRG_LEN)
    
# Will store id to later check correctness
ID = torchtext.data.Field(is_target=True, sequential=False)

In [10]:
train_dataset = DataFrameDataset(train_df, fields={'content_q':CONTEXT_Q,'answer':TRG, 'id':ID})
test_dataset = DataFrameDataset(test_df, fields={'content_q':CONTEXT_Q,'answer':TRG, 'id':ID})

In [11]:
if load:
    pass
else:
    # Build vocabulary from our data, target will have the same vocab as context + questions
    CONTEXT_Q.build_vocab(train_dataset, min_freq=150)
    TRG.build_vocab([''], min_freq=150)
    TRG.vocab = CONTEXT_Q.vocab
    
    
    with open("model/CONTEXT_Q.Field","wb+")as f:
        dill.dump(CONTEXT_Q,f)
    with open("model/TRG.Field","wb+")as f:
        dill.dump(TRG,f)
ID.build_vocab(list(train_df.id)+ list(test_df.id))

In [12]:
batch_size = 32
# Create iterators
train_iterator, test_iterator = DataFrameBucketIterator.splits((train_dataset, test_dataset), 
                                    batch_size = batch_size,
                                    device = device)

In [13]:
context_q_vocab = len(CONTEXT_Q.vocab)
target_vocab = len(TRG.vocab)
emb_size = 512
dim_feedforward = 512
num_head = 1

# Init model
model = BaselineModel(context_q_vocab, target_vocab, emb_size, num_head, dim_feedforward=dim_feedforward)
model.to(device)

BaselineModel(
  (context_emb): Embedding(9138, 512)
  (tgt_emb): Embedding(9138, 512)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropou

In [14]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'Model has a total of {count_trainable_parameters(model):,} of trainable parameters')

Model has a total of 22,462,386 of trainable parameters


In [15]:
optimizer = torch.optim.Adam(model.parameters(),lr=2e-4)
loss_func = nn.CrossEntropyLoss(ignore_index=1)

In [16]:
def train(model, iterator, optimizer, loss_func):
    """
    Runs training loop for whole dataset in iterator
    
    model - model to be trained
    iterator - data loader from which we take source and target
    optimizer - our optimizer
    loss_func - function which will compute loss
    return average loss
    """
    model.train() # Switch to train
    epoch_loss = [] # We will calculate cumulative loss
    
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        
        context = batch.content_q
        tgt = batch.answer
        
        output = model(context, tgt)
        tgt = tgt[1:].reshape(-1)
        output = output.view(-1, output.shape[-1]) 
        
        loss = loss_func(output, tgt)
        writer.add_scalar(f'Loss/train', loss, i)
        
        epoch_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()
    return epoch_loss

In [17]:
def evaluate(model, iterator, loss_func):
    """
    Runs an evaluation loop and returns average loss
    
    model - model to be evaluated
    iterator - data loader with validation set
    loss_func - function which will compute loss
    returns average loss
    """
    model.eval() # Switch to eval
    epoch_loss = 0 # We will calculate cumulative loss
    
    with torch.no_grad():
        to_return = []
        
        for i, batch in enumerate(iterator):
            optimizer.zero_grad()

            context = batch.content_q
            tgt = batch.answer
            
            output = model(context, teacher_forcing=0)
            
            softmaxed = nn.functional.softmax(output, dim=2)
            # store ids of batch and result
            to_return.append((softmaxed.topk(1)[1].squeeze().cpu().detach().numpy(), batch.id)) 
            
            tgt = tgt[1:].reshape(-1)
            output = output.view(-1, output.shape[-1]) 
            
            loss = loss_func(output, tgt)

            epoch_loss += loss.item()

            optimizer.step()
    return epoch_loss / len(iterator), to_return

## Training

In [ ]:
best_loss = float('inf')
epochs = 4
for epoch in range(epochs):
    train_loss = train(model, train_iterator, optimizer, loss_func)
    
    eval_loss, preds = evaluate(model, test_iterator, loss_func)
    
    # save "best" model
    if best_loss > eval_loss:
        best_loss = eval_loss
        torch.save(model.state_dict(), 'baseline.model')
    print(f"Epoch {epoch}. Train loss: {np.mean(train_loss)}. Eval loss: {eval_loss}")

## Evaluation

In [ ]:
_, preds = evaluate(model, test_iterator, loss_func)

In [ ]:
# flatten output
predictions = []
labels = []
for i in preds:
    for seq, tgt in zip(i[0].T, i[1]):
        if isinstance(seq, np.ndarray):
            predictions.append(seq)
        labels.append(tgt.T)

In [ ]:
def to_string(predictions):
    # Converting to string our predictions
    to_return = []
    for pred in predictions:
        result = []
        for word in pred:
            if word == 3: # If eos token - break
                break
            if word != 2: # If sos token - skip
                result.append(TRG.vocab.itos[word])
        to_return.append(' '.join(result))
    return to_return

In [ ]:
def check_correctness(predictions, labels, df):
    # Transform our predictions and check their correctness
    tokenizer = get_tokenizer("basic_english")
    answers = {}
    for i, row in df.iterrows():
        answers[row.id] = ' '.join(tokenizer(row.answer))
    
    correct = {}
    my_preds = {}
    for pred, tgt in zip(predictions, labels):
        correct[ID.vocab.itos[tgt]] = answers[ID.vocab.itos[tgt]] == pred
        my_preds[ID.vocab.itos[tgt]] = pred
        
    correct_preds = answers
    return correct, correct_preds, my_preds

In [ ]:
predictions = to_string(predictions)
correct, correct_preds, my_preds = check_correctness(predictions, labels, test_df)

In [ ]:
from evaluate import *

In [ ]:
# For the more representetive results we have taken script that squad owner's have written to check predictions

dataset = test_sq.data
preds = my_preds
na_probs = {k: 0.0 for k in preds}

qid_to_has_ans = make_qid_to_has_ans(dataset) 
has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]
exact_raw, f1_raw = get_raw_scores(dataset, preds)
exact_thresh = apply_no_ans_threshold(exact_raw, na_probs, qid_to_has_ans,
                                      1.0)
f1_thresh = apply_no_ans_threshold(f1_raw, na_probs, qid_to_has_ans,
                                   1.0)
out_eval = make_eval_dict(exact_thresh, f1_thresh)
if has_ans_qids:
    has_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=has_ans_qids)
    merge_eval(out_eval, has_ans_eval, 'HasAns')
if no_ans_qids:
    no_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=no_ans_qids)
    merge_eval(out_eval, no_ans_eval, 'NoAns')
print(json.dumps(out_eval, indent=2))

## Some samples of the results

In [ ]:
for i in range(5):
    choice = np.random.choice(list(my_preds))
    row = test_df[test_df.id == choice].iloc[0]
    print("Context: ", str(row.context))
    print()
    print("Question: ", str(row.content))
    print()
    if row.is_impossible:
        print("Impossible to answer")
    else:
        print("Answer: ", row.answer)
    print()
    if my_preds[choice]:
        print("Predicted answer: ", my_preds[choice])
    else:
        print("Predicted impossbile to answer")
    print("\n//////////////////// \n")